In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from data import SCRAPED_DIR
import datetime

LOAD DATA

In [ ]:
MAIN_INDEX = ["Date","Area","Name"]
SECONDARY_INDEX = ["Area","Name"]
YEARS_BACK_TO_CONSIDER = 5

stocks = pd.read_csv(
    f"{SCRAPED_DIR}/stocks.csv",
    dtype={
        "Date": "period[D]",
        "Open": "float",
        "High": "float",
        "Low": "float",
        "Close": "float",
        "Volume": "int",
        "Area": "category",
        "Name": "string",
    },
    index_col=MAIN_INDEX,
).sort_index().loc[datetime.date.today()-pd.DateOffset(years=YEARS_BACK_TO_CONSIDER):]

UNIQUE_AREAS = stocks.index.get_level_values("Area").unique()

In [ ]:
stocks

In [ ]:
# for area in UNIQUE_AREAS:
#     area_close = stocks
#     fig = plt.figure(figsize=(10,5))
#     fig.suptitle(f"Close price for {area} area")
#     area_close.plot(label=f"Close {area}", ax=fig.gca(), color="blue", legend=True)